# Repeating Every Assessment/Experiment With 2.5D Execution



##### <center> <i> <span style="color:green"> Before we start... </span> </i> </center>

As the title indicates, I'll be exploring results of using 2.5D classification, and comparing them with the replication of slices, in order to ascertain whether this approach is successful. 

Of course, these results will most likely be quite specific to my data set and methodologies, and will only represent a direct comparison and quality of my own methods.

In order to maintain consistent cross-analysis results contrasts, each experiment will retain its original train-test-validation split and ResNet setup (architecture, settings, etc.).

Additionally, I'll be avoiding some experimental comparisons, as they provided results that are guaranteed to be identical throughout both 2D and 2.5D classification. For example, I will not be using HRCT as the test split, nor testing whether the pure or conservative approach is best for best epoch choice. 

Still, all of this implies several code replications and re-executions. For this reason, I chose to display results in the form of a small report, with indexes and numbered titles, in order to facilitate comprehension and analysis.

# Table of Contents

- [1. Data Augmentation vs No Augmentation](#1-data-augmentation-vs-no-augmentation)
  - [1.1 Concept Overview](#11-concept-overview)
  - [1.2 Threshold Selection and Model Choice](#12-threshold-selection-and-model-choice)
  - [1.3 2.5D Execution of Data Augmentation vs No Augmentation](#13-25d-execution-of-data-augmentation-vs-no-augmentation)
  - [1.4 Comparison with 2D](#14-comparison-with-2d)

- [2. Patient-wise Classification Methods](#2-patient-wise-classification-methods)
  - [2.1 Concept Overview](#21-concept-overview)
  - [2.2 Threshold Selection and Model Choice](#22-threshold-selection-and-model-choice)
  - [2.3 2.5D Execution of Patient-wise Classification Methods](#23-25d-execution-of-patient-wise-classification-methods)
  - [2.4 Comparison with 2D](#24-comparison-with-2d)

- [3. Slice-level Feature-wise Classification](#3-slice-level-feature-wise-classification)
  - [3.1 Concept Overview](#31-concept-overview)
  - [3.2 Threshold Selection and Model Choice](#32-threshold-selection-and-model-choice)
  - [3.3 2.5D Execution of Slice-level Feature-wise Classification](#33-25d-execution-of-slice-level-feature-wise-classification)
  - [3.4 Comparison with 2D](#34-comparison-with-2d)


<a name="1-data-augmentation-vs-no-augmentation"></a>
## 1. Data Augmentation vs No Augmentation

<a name="11-concept-overview"></a>
### 1.1 Concept Overview  

<a name="12-threshold-selection-and-model-choice"></a>
### 1.2 Threshold Selection and Model Choice  

<a name="13-25d-execution-of-data-augmentation-vs-no-augmentation"></a>
### 1.3 2.5D Execution of Data Augmentation vs No Augmentation  

<a name="14-comparison-with-2d"></a>
### 1.4 Comparison with 2D  


<a name="2-patient-wise-classification-methods"></a>
## 2. Patient-wise Classification Methods

<a name="21-concept-overview"></a>
### 2.1 Concept Overview  

<a name="22-threshold-selection-and-model-choice"></a>
### 2.2 Threshold Selection and Model Choice  

<a name="23-25d-execution-of-patient-wise-classification-methods"></a>
### 2.3 2.5D Execution of Patient-wise Classification Methods  

<a name="24-comparison-with-2d"></a>
### 2.4 Comparison with 2D  


<a name="3-slice-level-feature-wise-classification"></a>
## 3. Slice-level Feature-wise Classification

<a name="31-concept-overview"></a>
### 3.1 Concept Overview  

<a name="32-threshold-selection-and-model-choice"></a>
### 3.2 Threshold Selection and Model Choice  

<a name="33-25d-execution-of-slice-level-feature-wise-classification"></a>
### 3.3 2.5D Execution of Slice-level Feature-wise Classification  

<a name="34-comparison-with-2d"></a>
### 3.4 Comparison with 2D  
